In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearnex import patch_sklearn

import time

start_time = time.time()

2023-06-30 15:02:59.296987: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-30 15:02:59.337157: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-30 15:02:59.338358: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 15:03:00.417945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'sklearnex'

In [ ]:

os.chdir("standard-ocr-dataset/data")





In [ ]:
def plot_images(images_arr, imageWidth, imageHeight):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img.reshape(imageWidth, imageHeight), cmap="gray")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

batch_size = 32
epochs = 50
IMG_HEIGHT = 28
IMG_WIDTH = 28

In [ ]:
def preprocessing_fun(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY, cv2.CV_8UC1)
    img = cv2.GaussianBlur(img, (3, 3), 0)
    img = img.reshape((28, 28, 1))
    thresh = cv2.adaptiveThreshold(img, 255, 1, 1, 11, 2)
    img = np.where(img > 140, 1, 0)
    img = img / 255



In [ ]:
# Enable Intel threading layer for better performance
patch_sklearn()



In [ ]:
augmented_image_gen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=2,
    width_shift_range=.1,
    height_shift_range=.1,
    zoom_range=0.1,
    shear_range=2,
    brightness_range=[0.9, 1.1],
    validation_split=0.2
)

normal_image_gen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2
)



In [ ]:
train_data_gen = augmented_image_gen.flow_from_directory(
    batch_size=batch_size,
    directory="./training_data",
    color_mode="grayscale",
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode="categorical",
    seed=65657867,
    subset='training'
)

val_data_gen = normal_image_gen.flow_from_directory(
    batch_size=batch_size,
    directory="./testing_data",
    color_mode="grayscale",
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode="categorical",
    seed=65657867,
    subset='validation'
)



In [ ]:
sample_training_images, _ = next(train_data_gen)
plot_images(sample_training_images[:7], IMG_WIDTH, IMG_HEIGHT)



In [ ]:
def define_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(36, activation='softmax'))
    return model



In [ ]:
model = define_model()



In [ ]:
# Intel-specific configuration options for TensorFlow
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
os.environ['TF_ENABLE_INTEL_OPS'] = '1'



In [ ]:
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

checkpoint_dir = 'path/to/checkpoints'  # Specify the checkpoint directory
checkpoint_path = os.path.join(checkpoint_dir, 'model_checkpoint.h5')

# Create the directory if it does not exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Define the callback for model checkpointing
checkpoint = ModelCheckpoint(
    checkpoint_path, monitor='val_loss', mode='min', save_best_only=True
)
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
my_callbacks = [EarlyStop_callback, checkpoint, lr]



In [ ]:
model.compile(optimizer=SGD(lr=0.01, momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['categorical_accuracy'])

history = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // batch_size,
    epochs=32,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // batch_size,
    callbacks=my_callbacks
)



In [ ]:
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(32)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



In [ ]:
normal_image_gen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True
)

test_data_gen = normal_image_gen.flow_from_directory(
    batch_size=5193,
    directory="./testing_data",
    color_mode="grayscale",
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode="categorical"
)



In [ ]:
from PIL import Image
test_images, test_labels = next(test_data_gen)
filenames = test_data_gen.filenames
test_pred = model.predict(test_images)



In [ ]:
true_labels = tf.argmax(test_labels, 1)
predicted_labels = tf.argmax(test_pred, 1)
confusion_matrix(true_labels, predicted_labels)



In [ ]:
img = cv2.imread('testing_data/N/28333.png', 0)
img = img / 255
img = cv2.resize(img, (28, 28))
img = img.reshape((1, 28, 28, 1))


In [ ]:
tf.argmax(model.predict(img), 1)


In [ ]:
elapsed_time = time.time() - start_time